In [139]:
from datetime import datetime
from ta import add_all_ta_features
import pandas as pd
import requests
import math
AV_KEY='5AK7ZPDAGCNO39B7'

In [149]:
STOCKS, CRYPTOS = ['500.PAR','IBM', 'BA'], ['ETH', 'BTC']
NAN_LIMIT = 25 #remove 

In [145]:
def GetStockPriceDF(symbol): 
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol='+symbol+'&outputsize=full&apikey='+AV_KEY
    print(url)
    r = requests.get(url)
    data = r.json()
    dic = data['Time Series (Daily)']
    df = pd.DataFrame.from_dict(dic, orient='index')
    df = df[['1. open', '2. high', '3. low', '4. close', '6. volume']]
    df = df.rename(columns={'1. open': 'open', '2. high': 'high', '3. low': 'low','4. close':'close', '6. volume':'volume'})
    df.index.name = 'date'
    return df

def GetCryptoPriceDF(symbol): 
    url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol='+symbol+'&market=CNY&apikey='+AV_KEY
    print(url)
    r = requests.get(url)
    data = r.json()
    dic = data['Time Series (Digital Currency Daily)']
    df = pd.DataFrame.from_dict(dic, orient='index')
    df = df[['1a. open (CNY)', '2a. high (CNY)', '3a. low (CNY)', '4a. close (CNY)', '5. volume']]
    df = df.rename(columns={'1a. open (CNY)': 'open', '2a. high (CNY)': 'high', '3a. low (CNY)': 'low', '4a. close (CNY)': 'close', '5. volume':'volume'})
    df.index.name = 'date'
    return df

def StripNaNCols(df):
    cols_to_keep = []
    for i in range(len(df.columns)):
        col = df.columns[i]
        nans = df[col].isna().sum() 
        if nans <= NAN_LIMIT:
            cols_to_keep.append(col)
    df = df[cols_to_keep]
    return df

def GetPriceDF(symbol):
    cols = ['open','high','low','close','volume']
    if symbol in CRYPTOS:
        df= GetCryptoPriceDF(symbol)
    else:
        df= GetStockPriceDF(symbol)
    df = df.dropna()
    df = df.sort_index(ascending = True)
    df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
    df = add_all_ta_features(df, open="open", high="high", low="low", close="close", volume="volume")
    df = StripNaNCols(df)
    df = df.dropna()
    df = df.add_prefix(symbol + '_')
    return df


In [150]:
df = GetPriceDF('500.PAR')

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=500.PAR&outputsize=full&apikey=5AK7ZPDAGCNO39B7


C:\Users\shave\AppData\Roaming\Python\Python37\site-packages\ta\trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
C:\Users\shave\AppData\Roaming\Python\Python37\site-packages\ta\trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


In [151]:
#print(df.head())
print(len(df.index))
print(len(df.columns))

2896
71


In [152]:
df.tail()

,500.PAR_open,500.PAR_high,500.PAR_low,500.PAR_close,500.PAR_volume,500.PAR_volume_adi,500.PAR_volume_obv,500.PAR_volume_cmf,500.PAR_volume_fi,500.PAR_volume_mfi,...,500.PAR_momentum_rsi,500.PAR_momentum_stoch,500.PAR_momentum_stoch_signal,500.PAR_momentum_wr,500.PAR_momentum_kama,500.PAR_momentum_roc,500.PAR_momentum_ppo,500.PAR_others_dr,500.PAR_others_dlr,500.PAR_others_cr
date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,75.0000,75.2339,74.6775,74.8157,34393,9.797195e+06,22332557,0.152932,6144.342134,52.074096,...,68.470122,91.296566,91.696742,-8.703434,73.870982,4.197237,9.941671,-0.008286,-0.008287,435.545455
2021-11-02,74.7440,75.2209,74.7440,75.2209,11617,9.808812e+06,22344174,0.157147,5939.037315,72.654528,...,70.498354,99.692954,96.996507,-0.307046,74.143360,3.877045,1.775356,0.541598,0.540136,438.445956
2021-11-03,75.2000,75.2309,75.0820,75.1559,3251,9.808788e+06,22340923,0.156556,5060.415556,72.696399,...,69.723529,97.563338,96.184286,-2.436662,74.312909,3.543058,-6.572471,-0.086412,-0.086450,437.980673
2021-11-04,75.8710,76.2770,75.8530,76.1082,50139,9.819005e+06,22391062,0.128043,11158.551862,73.796635,...,74.197835,96.018398,97.758230,-3.981602,74.733444,4.391945,-7.581450,1.267099,1.259139,444.797423
2021-11-05,76.1528,77.0700,76.1445,76.6530,11306,9.820123e+06,22402368,0.140863,10444.402853,66.442823,...,76.351017,90.872477,94.818071,-9.127523,75.211051,4.674457,-13.840109,0.715823,0.713273,448.697208


In [33]:

len(d.index)

100